In [1]:
import pandas as pd
from time import sleep
from slugify import slugify
import random
import datetime
import numpy as np

from selenium import webdriver
from selenium.common import exceptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException

from selenium_stealth import stealth

pd.set_option('display.max_colwidth', None)
%matplotlib inline

In [2]:
def generateDriver():
    options = webdriver.ChromeOptions() 
    options.add_argument("user-data-dir=selenium")
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage') 
    options.add_argument("start-maximized")
    options.add_argument("--disable-blink-features=AutomationControlled") 
    options.add_experimental_option('excludeSwitches', ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)
    driver = webdriver.Chrome(options = options)
    stealth(driver,
            languages=["en-US", "en"],
            vendor="Google Inc.",
            platform="Win32",
            webgl_vendor="Intel Inc.",
            renderer="Intel Iris OpenGL Engine")

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})") 
    driver.get('https://www.liveauctioneers.com/search/')
    return driver

driver = generateDriver()

In [19]:
slug = 'swastika'
# urls = pd.read_csv('data/urls/urls_' + slug + '.csv')
df = pd.read_csv('data/listings/listings_' + slug + '.csv')
urls = pd.DataFrame(df[df['img'].isna()].url)
i = 791
titles = []
dates = []
imgs = []
prices = []
auctioneer_urls = []
auctioneers = []
breadcrumbs = []
bidders = []
urlz = []

def scrape():
    global i
    global driver
    for url in urls.url[i:]:
        try:
            driver.get(url)
        except:
            print('crashed but restarting')
            driver = generateDriver()
            sleep(10)
            scrape()
                 
        print(url)
        
        try:
            show = driver.find_element(By.CSS_SELECTOR, 'button[class*=" FocusedItemSection__ViewItemDetailsButton-sc-"]')
            show.click()
            sleep(3)
        except exceptions.NoSuchElementException:
            sleep(1)
            pass

        try:
            title = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '[data-testid="pageTitle"] > div'))).get_attribute('innerHTML')
        except TimeoutException:
            title = ''
        
        try:
            auctioneer_url = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'a[class*=" AuctioneerInfo__NameLink-"]'))).get_attribute('href')
        except TimeoutException:
            auctioneer_url = ''
            
        try:
            auctioneer = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '[data-testid="itemPageSellerName"]'))).get_attribute('innerHTML')
        except TimeoutException:
            auctioneer = ''
            
        bcums = driver.find_elements(By.CSS_SELECTOR, '[data-testid="breadcrumb"] span')
        bcum = " > ".join([x.get_attribute('innerHTML') for x in bcums])
        
        try:
            img = WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.CSS_SELECTOR, '.item-image-primary img'))).get_attribute('src')
            if (img.startswith('data:image')):
                img = np.nan
        except TimeoutException:
            img = ''
            
        bidder = ''
        date = driver.find_element(By.CSS_SELECTOR,'[class*="ItemPageRight__StyledItemRight"] [data-testid="date-span"] span').get_attribute('innerHTML')
        
        if (driver.find_elements(By.CSS_SELECTOR,'[class*="BidHistoryTable__TableRowDiv"]:nth-child(2) > div:first-child')):
            bidder = driver.find_element(By.CSS_SELECTOR,
              '[class*="BidHistoryTable__TableRowDiv"]:nth-child(2) > div:first-child').get_attribute('innerHTML')
            
        if (driver.find_elements(By.CSS_SELECTOR, '[class^="ItemBiddingStatusRow__StyledStatus"] [class^="FormattedCurrency__StyledFormattedCurrency"]')):
            priceEl = driver.find_element(By.CSS_SELECTOR, '[class^="ItemBiddingStatusRow__StyledStatus"] [class^="FormattedCurrency__StyledFormattedCurrency"]')
            price = priceEl.get_attribute('innerHTML')
        else:
            price = np.nan

        titles.append(title)
        dates.append(date)
        auctioneer_urls.append(auctioneer_url)
        auctioneers.append(auctioneer)
        prices.append(price)
        imgs.append(img)
        breadcrumbs.append(bcum)
        bidders.append(bidder)
        urlz.append(url)

        loggedIn = driver.find_element(By.CSS_SELECTOR, '[data-testid="loggedInMenuMe"]')
        actions = ActionChains(driver)
        actions.move_to_element(loggedIn).move_by_offset(0, 40).click().perform()

        i += 1
        print(i)
    print('all done')

scrape()


all done


In [18]:
# First time
# df = pd.DataFrame(list(zip(titles, dates, prices, breadcrumbs, bidders, auctioneers, imgs, urlz, auctioneer_urls)),
#                   columns=['title', 'date', 'price', 'cats', 'final_bidder', 'auctioneer', 'img', 'url', 'auctioneer_url'])
# df.to_csv('data/listings/listings_' + slug + '.csv', index = False)
# len(df)

# After
df1 = pd.read_csv('data/listings/listings_' + slug + '.csv')
df2 = pd.DataFrame(list(zip(titles, dates, prices, breadcrumbs, bidders, auctioneers, imgs, urlz, auctioneer_urls)),
                  columns=['title', 'date', 'price', 'cats', 'final_bidder', 'auctioneer', 'img', 'url', 'auctioneer_url'])

df3 = pd.concat([df1, df2]).drop_duplicates().reset_index(drop = True)
df3.to_csv('data/listings/listings_' + slug + '.csv', index = False)
len(df3)


6477

In [20]:
# Fill in nans
df3 = pd.read_csv('data/listings/listings_swastika.csv')
df4 = df3.loc[df3.notnull().sum(1).groupby([df3.url, df3.img]).idxmax()]
df4[df4['img'].isna()]
df4.to_csv('data/listings/listings_' + slug + '.csv', index = False)

In [7]:
# Consolidate listings
df1 = pd.read_csv('data/listings/listings_swastika.csv')
df2 = pd.read_csv('data/listings/listings_not_swastika.csv')
df3 = pd.concat([df1, df2], ignore_index = True).drop_duplicates('url')
df3.to_csv('data/listings/listings_all.csv', index = False)